In [1]:
import sys
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
df = pd.read_csv('gdrive/My Drive/fer2013.csv')
print ((df['Usage']).unique())


from google.colab import drive
drive.mount('/content/gdrive')

['Training' 'PublicTest' 'PrivateTest']
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
x_train, y_train, x_test, y_test = [], [], [], []

for index, row in df.iterrows():
  val = row['pixels'].split(" ")
  try:
    if 'Training' in row['Usage']:
      x_train.append(np.array(val, 'float32'))
      y_train.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
      x_test.append(np.array(val, 'float32'))
      y_test.append(row['emotion'])
  except:
    print (f"error occured at index:{index} and row:{row}")

print (x_train[0:2])
print (x_test[0:2])
print (y_train[0:2])
print (y_test[0:2])

x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
[0, 0]
[0, 1]


In [0]:
# Normalizing
x_train -= np.mean(x_train, axis=0)
x_train /= np.std(x_train, axis=0)

x_test -= np.mean(x_test, axis=0)
x_test /= np.std(x_test, axis=0)

In [0]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x_train = x_train.reshape(x_train.shape[0], width, height, 1)
x_test = x_test.reshape(x_test.shape[0], width, height, 1)

y_train = np_utils.to_categorical(y_train, num_classes=num_labels)
y_test = np_utils.to_categorical(y_test, num_classes=num_labels)

In [6]:
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1:])))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())

model.add(Dense(2*2*2*2*num_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2*2*2*2*num_features, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          shuffle=True)

fer_json = model.to_json()
with open("fer.json", "w") as json_file:
  json_file.write(fer_json)

model.save_weights("fer.h5")


W0905 15:10:59.669652 140340527249280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0905 15:10:59.689970 140340527249280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0905 15:10:59.693472 140340527249280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0905 15:10:59.732160 140340527249280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0905 15:10:59.736069 140340527249280 deprecation_wrapp

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 25s 871us/step - loss: 1.7430 - acc: 0.2848 - val_loss: 1.6144 - val_acc: 0.3555
Epoch 2/100
28709/28709 [==============================] - 23s 787us/step - loss: 1.5259 - acc: 0.3970 - val_loss: 1.4259 - val_acc: 0.4475
Epoch 3/100
28709/28709 [==============================] - 23s 784us/step - loss: 1.4110 - acc: 0.4472 - val_loss: 1.3115 - val_acc: 0.4935
Epoch 4/100
28709/28709 [==============================] - 22s 782us/step - loss: 1.3422 - acc: 0.4847 - val_loss: 1.2696 - val_acc: 0.5038
Epoch 5/100
28709/28709 [==============================] - 23s 790us/step - loss: 1.2934 - acc: 0.5010 - val_loss: 1.2712 - val_acc: 0.5163
Epoch 6/100
28709/28709 [==============================] - 23s 785us/step - loss: 1.2610 - acc: 0.5131 - val_loss: 1.2203 - val_acc: 0.5249
Epoch 7/100
28709/28709 [==============================] - 23s 784us/step - loss: 1.2275 - acc: 0.5298 - val_lo